# 將預測模型套用在圖片上

In [ ]:
import numpy as np
from PIL import Image
import os
import cv2
from tensorflow import keras


images = []
folder = './'
filename = 'test.jpg'
chosen_model = 'Densenet'
img_path = os.path.join(folder, filename)
img = Image.open(img_path)
img = img.convert('L')  # Ensure the image is in gray format
img = img.resize((128, 128))  # Resize image to a fixed size
img_array = np.array(img)
x_images = img_array.astype('float32') / 255.0
x_images = x_images.reshape((1, 128, 128))


# choose model 
if chosen_model =='Densenet':
    new_model = keras.models.load_model('./train_model_code/densenet_model/saved_model1.keras')
elif chosen_model == 'Autoencoder':
    new_model = keras.models.load_model('save_model.keras')
    encoded_imgs = new_model.encoder(x_images).numpy()
    result = new_model.decoder(encoded_imgs).numpy()

result = new_model.predict(x_images)

result = (result * 255).astype(np.uint8)

# Remove the batch dimension and reshape if necessary
result = result.reshape((128, 128))
result = cv2.resize(result, (256, 256))
# Convert to an image and save
output_image = Image.fromarray(result)
output_image.save("outcome.jpg")

# 嘴唇上色

In [2]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Face Mesh and drawing utils
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

lip_u = [0, 37, 39, 185, 80, 81, 82, 13, 312, 311, 310, 409, 270, 269, 267]
lip_l = [14, 87, 178, 88, 95, 146, 91, 181, 84, 17, 314, 405, 321, 375, 324, 318, 402, 317]

# Path to the image
image_path = './256img_lst/006.jpg'

# Read the image
image = cv2.imread(image_path)
if image is None:
    print(f"Failed to load image: {image_path}")
    exit()


# Define the makeup effect function
def apply_lipstick(image, landmarks, color,const = 0.9 ):
    lips_points_u = np.array([landmarks[i] for i in lip_u])  # Upper lip points
    lips_points_l = np.array([landmarks[i] for i in lip_l])  # Lower lip points

    # Get color data for upper lip points
    upper_lip_colors = [image[y, x] for (x, y) in lips_points_u]

    # Get color data for lower lip points
    lower_lip_colors = [image[y, x] for (x, y) in lips_points_l]
    
    mask = image.copy()
    cv2.fillPoly(mask, [lips_points_u], color)
    cv2.fillPoly(mask, [lips_points_l], color)
    # Convert the image to grayscale
    gray_mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)

# Parameters for the GuidedFilter
    radius = 10         # Radius of the guided filter
    eps = 0.1         # Regularization parameter (epsilon)

    guided_filter = cv2.ximgproc.createGuidedFilter(guide=gray_mask, radius=radius, eps=eps)
    filtered_mask = guided_filter.filter(mask)

    result = cv2.addWeighted(image, const, filtered_mask, 1-const, 0)

    return result

# Create trackbars to adjust lip color
def nothing(x):
    pass

cv2.namedWindow('Trackbars')
cv2.createTrackbar('R', 'Trackbars', 0, 255, nothing)
cv2.createTrackbar('G', 'Trackbars', 0, 255, nothing)
cv2.createTrackbar('B', 'Trackbars', 0, 255, nothing)
cv2.createTrackbar('level', 'Trackbars', 0, 100, nothing)



# Detect face in the image
results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

if results.multi_face_landmarks:
    for face_landmarks in results.multi_face_landmarks:
        height, width, _ = image.shape
        landmarks = [(int(point.x * width), int(point.y * height)) for point in face_landmarks.landmark]

while True:
    # Get the current positions of the trackbars
    r = cv2.getTrackbarPos('R', 'Trackbars')
    g = cv2.getTrackbarPos('G', 'Trackbars')
    b = cv2.getTrackbarPos('B', 'Trackbars')
    const = cv2.getTrackbarPos('level', 'Trackbars')
    const = -(float(const)-50)/500+0.9
    img = np.zeros((100, 100, 3), np.uint8) 
    img[:] = [b,g,r]
    # Apply the makeup effect with the current color
    if results.multi_face_landmarks:
        output_image = apply_lipstick(image.copy(), landmarks, (b, g, r),const)
    else:
        output_image = image.copy()

    # Display the result
    cv2.imshow('Lipstick Application', output_image)
    cv2.imshow('color plate',img)
    # Exit on 'esc' key press
    if cv2.waitKey(1) & 0xFF == 27:
        break

cv2.destroyAllWindows()
face_mesh.close()

error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window_w32.cpp:2564: error: (-27:Null pointer) NULL window: 'Trackbars' in function 'cvGetTrackbarPos'


# 眼妝

In [2]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Face Mesh and drawing utils
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles


# Define the makeup effect function
def apply_eyeshade(image, mask, color,region_rate ,const = 0.9 ):
    
    gray_mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    mask1 = np.all(mask >255-region_rate*255/100, axis=-1)

    mask2 = image.copy()
    mask2[mask1] = gray_mask[mask1][:, np.newaxis] *color/255

# Parameters for the GuidedFilter
    radius = 10         # Radius of the guided filter
    eps = 0.1         # Regularization parameter (epsilon)

    guided_filter = cv2.ximgproc.createGuidedFilter(guide=gray_mask, radius=radius, eps=eps)
    filtered_mask = guided_filter.filter(mask2)
    result = cv2.addWeighted(image, const, filtered_mask, 1-const, 0)

    return result

# Create trackbars to adjust lip color
def nothing(x):
    pass

cv2.namedWindow('Trackbars')
cv2.createTrackbar('R', 'Trackbars', 0, 255, nothing)
cv2.createTrackbar('G', 'Trackbars', 0, 255, nothing)
cv2.createTrackbar('B', 'Trackbars', 0, 255, nothing)
cv2.createTrackbar('level', 'Trackbars', 0, 100, nothing)
cv2.createTrackbar('region_rate', 'Trackbars', 0, 100, nothing)

# Path to the image
image_path = './256img_lst/006.jpg'
predicted_result_path ='outcome.jpg'

# Read the image
image = cv2.imread(image_path)
predicted_result = cv2.imread(predicted_result_path)
if image is None:
    print(f"Failed to load image: {image_path}")
    exit()

if predicted_result is None:
    print(f"Failed to load image: {predicted_result_path}")
    exit()

# Detect face in the image
results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

if results.multi_face_landmarks:
    for face_landmarks in results.multi_face_landmarks:
        height, width, _ = image.shape
        landmarks = [(int(point.x * width), int(point.y * height)) for point in face_landmarks.landmark]

while True:
    # Get the current positions of the trackbars
    r = cv2.getTrackbarPos('R', 'Trackbars')
    g = cv2.getTrackbarPos('G', 'Trackbars')
    b = cv2.getTrackbarPos('B', 'Trackbars')
    const = cv2.getTrackbarPos('level', 'Trackbars')
    region_rate = cv2.getTrackbarPos('region_rate', 'Trackbars')


    const = -(float(const)-50)/500+0.9
    img = np.zeros((100, 100, 3), np.uint8) 
    img[:] = [b,g,r]
    # Apply the makeup effect with the current color
    if results.multi_face_landmarks:
        output_image = apply_eyeshade(image.copy(),predicted_result.copy() , (b, g, r),region_rate,const)
    else:
        output_image = image.copy()
    
    # Display the result
    cv2.imshow('Lipstick Application', output_image)
    cv2.imshow('color plate',img)

    # Exit on 'esc' key press
    if cv2.waitKey(1) & 0xFF == 27:
        break
cv2.destroyAllWindows()
face_mesh.close()

In [5]:
import cv2
import mediapipe as mp
import numpy as np
rwmask_path ='./pymatting_outcome_rw/006.jpg'
mask = cv2.imread(rwmask_path)

cv2.imshow('Modified Mask', mask)
cv2.waitKey(0)
cv2.destroyAllWindows()
print(mask)


[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]


In [4]:
mask = cv2.imread(rwmask_path)
cv2.imshow('or', mask)
mask1 = np.all(mask != (0, 0, 0), axis=-1)
gray_mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
mask2 = image.copy()

reshaped_masked_array = gray_mask[mask1][:, np.newaxis]  # Shape (4825, 1)
color = np.array([0,0,127])
# Repeat the (1, 1, 1) vector to match the shape (4825, 3)
mask2[mask1] = gray_mask[mask1][:, np.newaxis] *color/255  # Broadcasting will happen here
print(mask2)

cv2.imshow('Modified Mask', mask2)
cv2.waitKey(0)
cv2.destroyAllWindows()
# Convert the image to grayscale

[[[213 229 246]
  [213 229 246]
  [215 228 244]
  ...
  [ 24  41  67]
  [ 26  46  71]
  [ 29  49  74]]

 [[211 227 244]
  [211 227 244]
  [212 227 243]
  ...
  [ 25  42  68]
  [ 28  48  73]
  [ 31  51  76]]

 [[209 225 242]
  [209 225 242]
  [211 226 242]
  ...
  [ 27  44  70]
  [ 30  50  75]
  [ 34  54  79]]

 ...

 [[ 90 102 120]
  [ 86  98 116]
  [ 78  92 111]
  ...
  [ 43  54  81]
  [ 42  53  80]
  [ 40  51  78]]

 [[ 91 103 121]
  [ 84  96 114]
  [ 72  86 105]
  ...
  [ 42  53  80]
  [ 41  52  79]
  [ 39  50  77]]

 [[ 91 103 121]
  [ 82  94 112]
  [ 68  82 101]
  ...
  [ 42  53  80]
  [ 41  52  79]
  [ 39  50  77]]]


In [3]:
# Path to the image
image_path = './256img_lst/006.jpg'
rwmask_path ='./pymatting_outcome_rw/006.jpg'

# Read the image
image = cv2.imread(image_path)

mask = cv2.imread(rwmask_path)
cv2.imshow('or', mask)
gray_mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
mask1 = np.all(mask >127, axis=-1)

mask2 = image.copy()
color = np.array([0,0,127])
mask2[mask1] = gray_mask[mask1][:, np.newaxis] *color/255

# reshaped_masked_array = gray_mask[mask1][:, np.newaxis]  # Shape (4825, 1)
# # Repeat the (1, 1, 1) vector to match the shape (4825, 3)
# mask2[mask1] = gray_mask[mask1][:, np.newaxis] *color/255  # Broadcasting will happen here
print(mask2.shape)

cv2.imshow('Modified Mask', mask2)
cv2.waitKey(0)
cv2.destroyAllWindows()

(256, 256, 3)
